# Dimensionnement d'une cuve

In [102]:
import numpy as np
import pandas as pd
import altair as alt

## Méthode

On utilise un modèle de pluviométrie statique : on sait exactement, et à l'avance, quand la pluie tombera et en quelle quantité. 
Par ailleurs la consommation est également prédictible jour par jour. 

Pour construire le modèle nous utilisons la fiche climatique de Clermont-Ferrand qui donne, pour chaque mois, la hauteur cumulée tombée, et le nombre de jours où il a plu. 

Comme on ne sait pas quel jour il a plu, on distribue aléatoirement les jours de pluie dans le mois, et on considère qu'il tombe toujours la même quantité. 

## Modèle de pluviométrie

### Pluviométrie locale

Considérons la fiche climatique de Clermont-Ferrand. 

In [103]:
moyenne_clermont = pd.read_csv('../data/fiche_climat_clermont.csv', sep=',', )
precipitations = pd.read_csv('../data/daily_rain_clermont.csv',
                              sep=',').replace(np.nan, 0)
moyenne_clermont.head()

,mois,hauteur,nb_jours
0,janvier,26.6,6.4
1,fevrier,18.7,5.0
2,mars,26.1,6.5
3,avril,51.1,8.3
4,mai,66.5,9.4


Et visualisons-la :

In [104]:
mois = ['janvier', 'fevrier', 'mars', 'avril', 'mai', 'juin', 'juillet', 'aout', 'septembre', 'octobre', 'novembre', 'decembre']
alt.Chart(moyenne_clermont).mark_bar().encode(
    alt.X('mois:N', title='Mois', sort=mois),
    alt.Y('hauteur', title='Hauteur'),
    tooltip=['mois', 'hauteur']
) | alt.Chart(moyenne_clermont).mark_bar(color='orange').encode(
    alt.X('mois:N', title='Mois', sort=mois),
    alt.Y('nb_jours', title='Nombre de jours de pluie'),
    tooltip=['mois', 'nb_jours']
)


alt.HConcatChart(...)

A partir de ces données, on simule une année de pluviométrie avec les réserves indiquées plus haut. 
Le résultat est donné ci-dessous

In [105]:

alt.Chart(precipitations).mark_point().encode(
    x=alt.X('doy:T', 
            title = 'Jour de l\'annee'),
    y=alt.Y('mm:Q', title = 'Hauteur des précipitations')
).properties(width=450).interactive()

# NB : Altair infere le mois de l'annee en Anglais

alt.Chart(...)

Comme prévu, ce modèle n'est pas représentatif de la réalité, car les pluies sont bien trop homogènes en hauteur. 
On en tire immédiatement l'eau disponible à chaque pluie grâce à la surface du toit. 

## Consommation

### Typologie des consommations

On distingue ici trois sortes de besoins : 

* irrigation (potager)
* agricole (machines)
* domestiques

Dans un premier temps, nous ne considérerons que les besoins d'irrigation. 

### Besoins potager

Le potager à irriguer a une surface de $100 m^2$ . 

Pour simplifier les calculs, on ne prendra que trois valeurs de besoins en eau du potager : 

* D'octobre à avril il n'y a pas besoin d'irrigation.
* Entre juin et août : irrigation maximale.
* En mai et septembre, irrigation modérée. 

Pour simplifier, on considérera que l'irrigation est de $20L/m^2$ tous les trois jours *sans paillage* au plus fort de l'été. 



In [106]:
besoins = pd.read_csv('../data/consommation_potager.csv', sep=',')
besoins.head()

,doy,julianDay,conso_potager,conso_paillage
0,01-01,1,0.0,0.0
1,01-02,2,0.0,0.0
2,01-03,3,0.0,0.0
3,01-04,4,0.0,0.0
4,01-05,5,0.0,0.0


In [107]:
alt.Chart(besoins).mark_line().encode(
    x=alt.X('monthdate(doy):T', title="Jour de l'année"),
    y=alt.Y('conso_potager:Q', title='Consommation (L/m2)'),
    tooltip=[alt.Tooltip("conso_potager:Q")]
).properties(width=600)

alt.Chart(...)

## Dimension minimale de la cuve

Quels sont les besoins totaux ? 

In [108]:
print(np.sum(besoins['conso_potager']*100)) 


81677.0


Le potager a donc besoin, en tout, de 82 $m^3$ au cours d'une année. 
Mais une partie de ces besoins est fournie par la pluie, lorsqu'elle tombe. 

En première approximation, considérons qu'à chaque fois qu'il pleut les besoins sont nuls pour la journée, sans influer sur ceux du lendemain. 


In [109]:
df = pd.merge(precipitations, besoins, on='doy')
df.head()

,doy,mm,julianDay,conso_potager,conso_paillage
0,01-01,0.0,1,0.0,0.0
1,01-02,0.0,2,0.0,0.0
2,01-03,0.0,3,0.0,0.0
3,01-04,0.0,4,0.0,0.0
4,01-05,0.0,5,0.0,0.0


In [110]:
df['irrigationSurfacique'] = df.conso_potager - df.mm
df.irrigationSurfacique[df['irrigationSurfacique']<0]=0

/var/folders/fj/3hr2gzvj10z18jrd95l4p4xw0000gn/T/ipykernel_1565/2816986552.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.irrigationSurfacique[df['irrigationSurfacique']<0]=0


In [111]:
alt.Chart(df).mark_line().encode(
    x = alt.X('monthdate(doy)', title="Jour de l'année"),
    y = alt.Y('conso_paillage', title="Besoins en irrigation")
).properties(width=450).interactive()

alt.Chart(...)

In [112]:
# A partir des besoins et consommations par unité de surface,
# On obtient les besoins et consommations pour l'ensemble du potager. 

df['besoinsTotaux'] = df.irrigationSurfacique * 100
df['apportsTotaux'] = df.mm*200
# besoins totaux donne la totalité du besoin pour le potager, soit sur 100 m2

On accède donc, via la surface du potager et celle du toit, à l'ensemble des besoins et l'ensemble des apports quotidiens. 


### Dimension de la cuve

Simulons une cuve de volume infini, qui se remplit avec les précipitations et se vide via les utilisations, et suivons son volume. 

In [113]:
#initialisation 
cuve = [0]
# remplissage au cours de l'annee
for index in range(len(df)-1):
    cuve.append(cuve[index] + df['apportsTotaux'].iloc[index] - df['besoinsTotaux'].iloc[index])
#conversion en df
cuve = pd.DataFrame(cuve)
cuve = pd.concat([df['doy'], cuve], axis=1)
cuve = cuve.set_axis(['doy', 'vol'], axis=1, copy=False)

In [114]:
# Visualisation graphique
alt.Chart(cuve).mark_line().encode(
    x=alt.X('doy:T',title="Jour de l'année"),
    y=alt.Y('vol',title="Volume de la cuve"),
    tooltip=['vol', 'doy']
).properties(
    width = 400
).interactive()

alt.Chart(...)

La cuve, initialement vide, se remplit pendant les mois d'hiver. 
Intuitivement sur la partie 01 mai - 31 oct, il suffit de prendre le maximum et le minimum locaux pour en déduire la réserve qui doit être atteinte. 

In [115]:
dcuve = cuve.loc[120:270]
print("Volume max : ", np.max(dcuve['vol']))
print("Volume min : ", np.min(dcuve['vol']))
print('Volume nécessaire : ', (np.max(dcuve['vol'])-np.min(dcuve['vol'])))

Volume max :  36454.0
Volume min :  23637.0
Volume nécessaire :  12817.0


On a donc besoin d'une cuve de 12 817 L. 
Vérifions le comportement d'une telle cuve. 

In [116]:
dcuve = cuve
dcuve['vol'] = 0
volmax = 13000
for index in (range(len(dcuve['vol'])-1)):
    dcuve['vol'].iloc[index+1] = dcuve['vol'].iloc[index] + df['apportsTotaux'].iloc[index] - df['besoinsTotaux'].iloc[index]
    dcuve['vol'].iloc[index+1] = min(dcuve['vol'][index+1], volmax)


/var/folders/fj/3hr2gzvj10z18jrd95l4p4xw0000gn/T/ipykernel_1565/2298151465.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dcuve['vol'].iloc[index+1] = dcuve['vol'].iloc[index] + df['apportsTotaux'].iloc[index] - df['besoinsTotaux'].iloc[index]
/var/folders/fj/3hr2gzvj10z18jrd95l4p4xw0000gn/T/ipykernel_1565/2298151465.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dcuve['vol'].iloc[index+1] = min(dcuve['vol'][index+1], volmax)


In [117]:
alt.Chart(dcuve).mark_line().encode(
    x='doy:T',
    y='vol'
).properties(
    width=500
).interactive()

alt.Chart(...)

## Dimensions d'une cuve pour un potager avec paillage

Le paillage diminue les besoins en eau du potager de 66%. 
Qu'en est-il du volume nécessaire de la cuve ? 
Pour répondre à cette question nous reproduisons la manipulation faite plus haut. 

In [118]:
# Ajout de besoins en irrigation avec paillage
df['irrigationPaillage'] = df.conso_potager - df.mm
df.irrigationSurfacique[df['irrigationPaillage']<0]=0
df['besoinsPaillage'] = df.irrigationPaillage * 100
# df.apportsTotaux reste le meme. 

# Creation de la seconde cuve
#initialisation 
cuvePaillage = [0]
# remplissage au cours de l'annee
for index in range(len(df)-1):
    cuvePaillage.append(cuvePaillage[index] + df['apportsTotaux'].iloc[index] - df['besoinsPaillage'].iloc[index])
#conversion en df
cuvePaillage = pd.DataFrame(cuvePaillage)
cuvePaillage = pd.concat([df['doy'], cuvePaillage], axis=1)
cuvePaillage = cuvePaillage.set_axis(['doy', 'vol'], axis=1, copy=False)


# Visualisation graphique
alt.Chart(cuvePaillage).mark_line().encode(
    x=alt.X('doy:T',title="Jour de l'année"),
    y=alt.Y('vol',title="Volume de la cuve"),
    tooltip=['vol', 'doy']
).properties(
    width = 400
).interactive()


/var/folders/fj/3hr2gzvj10z18jrd95l4p4xw0000gn/T/ipykernel_1565/4036609774.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.irrigationSurfacique[df['irrigationPaillage']<0]=0


alt.Chart(...)

In [119]:
#Nouveaux besoins 
dcuve = cuvePaillage.loc[170:270]
print("Volume max : ", np.max(dcuve['vol']))
print("Volume min : ", np.min(dcuve['vol']))
print('Volume nécessaire : ', (np.max(dcuve['vol'])-np.min(dcuve['vol'])))

Volume max :  55762.0
Volume min :  43957.0
Volume nécessaire :  11805.0


In [120]:
# Ce n'est pas une copie, on a maintenant deux références à la même variable
# donc mutation sur dcuve <=> mutation sur cuvePaillage
# donc aucun intérêt de renommer
dcuve = cuvePaillage
dcuve['vol'] = 0
volmax = 10056
for index in (range(len(dcuve['vol'])-1)):
    dcuve['vol'].iloc[index+1] = dcuve['vol'].iloc[index] + df['apportsTotaux'].iloc[index] - df['besoinsPaillage'].iloc[index]
    dcuve['vol'].iloc[index+1] = min(dcuve['vol'][index+1], volmax)
alt.Chart(dcuve).mark_line().encode(
    x='doy:T',
    y='vol'
).properties(
    width=500
).interactive()


/var/folders/fj/3hr2gzvj10z18jrd95l4p4xw0000gn/T/ipykernel_1565/2194624315.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dcuve['vol'].iloc[index+1] = dcuve['vol'].iloc[index] + df['apportsTotaux'].iloc[index] - df['besoinsPaillage'].iloc[index]
/var/folders/fj/3hr2gzvj10z18jrd95l4p4xw0000gn/T/ipykernel_1565/2194624315.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dcuve['vol'].iloc[index+1] = min(dcuve['vol'][index+1], volmax)


alt.Chart(...)

On voit ici qu'on est parvenu à réduire les besoins de seulement deux mètres cubes, ce qui n'est pas intuitif : en effet, on a divisé les besoins par trois. 
La raison est sans doute dans l'hypothèse qu'une pluie réduit les besoins à zéro seulement pour le jour où il pleut. Or une pluie de 30mm en juillet permet d'éviter d'arroser pendant plusieurs jours. 


## PLUS ULTRA

Bouts de code à retirer et problèmes

### tentative d'appel de fonction dans un dossier parent


La prochaine étape pour se débarrasser du fichier .csv de pluviométrie quotidienne est de créer une fonction qui le fait à partir de la fiche climatique. 

In [121]:
# Il faut que tu importes depuis un endroit connu
# Ton projet définit la _library_ `recuperation_eau_pluie`,
# et l'installe automatiquement de façon éditable
# (on en discute à l'occasion si ça t'intéresse)
from recuperation_eau_pluie import test_function

In [122]:
test_function(0)

hello world !


In [123]:
# Pandas ne sait pas de quelle année on parle, il ne va pas l'inventer... (année bissextile?)
dcuve['doy'] = pd.to_datetime(dcuve['doy'].apply(lambda d: f"2020-{d}"))

In [125]:
# dt.month est un entier
dcuve.loc[dcuve['doy'].dt.month == 1]

,doy,vol
0,2020-01-01,0
1,2020-01-02,0
2,2020-01-03,0
3,2020-01-04,0
4,2020-01-05,0
5,2020-01-06,0
6,2020-01-07,0
7,2020-01-08,0
8,2020-01-09,0
9,2020-01-10,0


In [126]:
pluvio = [[], []]
# NB : moyenne_clermont est le resultat de pd.read_csv
for index in range(len(moyenne_clermont['mois'])):
    # Nombre de jours ou il pleut, loi normale
    mu_freq = moyenne_clermont['nb_jours'][index]
    sigma_freq = mu_freq/2

    #exemple : cuve

    
    # Définition
    mu_p = moyenne_clermont['hauteur'][index]
    sigma_p = mu_p/10
    # 

In [128]:
# Évite de mettre des cellules qui modifient la donnée et d'autres qui l'affiche
# Sinon affichage => modification en effet indésirable
precipitations['cumul']=np.cumsum(precipitations.mm)

In [132]:
alt.Chart(precipitations).mark_area(
    color="lightblue",
    interpolate='step-after',
    line=True
).encode(
    x=alt.X('monthdate(doy):T', title = "Jour de l'année"),
    y=alt.Y('cumul', title = "Pluie cumulée depuis le début de l'année")
).properties(width=600)

alt.Chart(...)

In [135]:
# Calcul du volume d'eau contenu dans la cuve

# Là on en reparle mais c'est pas du tout optimal.
# En Python les boucles sont etrêmement lentes. Il aurait fallut faire ça avec Pandas directement.

cuve = [0]
for p in [0, 1000, 3000, 20000, 0]:
    cuve = [p]
    for index in range(len(df)):
        cuve.append(cuve[index-1] + df['apportsTotaux'].iloc[index] - df['besoinsTotaux'].iloc[index])
    print('cuve ', p, ', min = ', min(cuve), ', max = ', max(cuve))
    print('****')
    print('diff = ', abs(min(cuve)-max(cuve)))
    print('****')

    

cuve  0 , min =  0 , max =  29186.0
****
diff =  29186.0
****
cuve  1000 , min =  1000 , max =  30186.0
****
diff =  29186.0
****
cuve  3000 , min =  3000 , max =  32186.0
****
diff =  29186.0
****
cuve  20000 , min =  20000 , max =  49186.0
****
diff =  29186.0
****
cuve  0 , min =  0 , max =  29186.0
****
diff =  29186.0
****


In [136]:
cuve = [0]

for index in range(len(df)):
    cuve.append(cuve[index-1] + df['apportsTotaux'].iloc[index] - df['besoinsTotaux'].iloc[index])

cuve=pd.DataFrame(list(zip(cuve, range(len(cuve)))), 
                  columns=['vol', 'doy'])

In [137]:
cuve

,vol,doy
0,0.0,0
1,0.0,1
2,0.0,2
3,0.0,3
4,0.0,4
...,...,...
361,25189.0,361
362,29186.0,362
363,25189.0,363
364,29186.0,364
